In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Dense
from tensorflow.keras.regularizers import l2

In [3]:
def load_df(company_code):

    # Load the Excel file and read Data from the file
    file_path = company_code + '.xlsx'
    sheet_name = company_code
    data = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')

    # Remove rows with any NaN values
    data = data.dropna()
    # Reset the index of the DataFrame and drop the old index
    data = data.reset_index(drop=True)
    return data

def repeat_quaterly_data(quaterly_data, daily_data):
    quaterly_data.index = pd.to_datetime(quaterly_data.index)
    daily_data.index = pd.to_datetime(daily_data.index)

    # Resample the quaterly data to daily frequency
    quaterly_data.index = quaterly_data.index + pd.DateOffset(days=1)
    # Using Forward Fill to Resample the Quaterly Data
    # daily_quaterly_data = quaterly_data.resample('D').ffill()
    # quaterly_data.index = quaterly_data.index - pd.offsets.MonthEnd(1) - pd.DateOffset(days=1)

    aligned_quaterly_data = quaterly_data.reindex(daily_data.index, method='ffill')
    aligned_quaterly_data = aligned_quaterly_data.dropna()
    return aligned_quaterly_data

# Function to create sequence
# Need to define the sequence length, e.g. using 4 quaters to predict the next quater
def create_sequences(features, targets, seq_length):
    xs = []
    ys = []
    for i in range(len(features)-seq_length):
        x = features[i:(i+seq_length)]
        y = targets.iloc[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

def get_historical_data(ticker, start_date, end_date):
    data = yf.download(ticker,start=start_date, end=end_date)
    return data

# Function to fetch Historical Price data and compute returns
def get_historical_returns(ticker, start_date, end_date):
    data = yf.download(ticker,start=start_date, end=end_date)

    # Calculate daily returns
    # data['Returns'] = data['Close'].pct_change()
    # data = data.dropna()

    # Monthly Return
    monthly_data = data['Close'].resample('M').last()
    monthly_returns = monthly_data.pct_change()
    monthly_returns = monthly_returns.dropna()

    # return data['Returns']
    return monthly_returns

In [4]:
# Setup TimeFrame and Target Firm
ticker = 'MSFT'
company_code = 'MSFT-US'
start_date = '2017-08-30'
end_date = '2023-09-30'

# Process Ratio Data
data = load_df(company_code)
data = data.set_index('Date').T
data.index = pd.to_datetime(data.index, format='%b \'%y')
data.index = data.index + pd.offsets.MonthEnd()
ratio_data = data.apply(pd.to_numeric)

# Select a few columns
pe_column = 'Price/Earnings'
pb_column = 'Price/Book Value'
roa_column = 'Return on Assets'
roe_column = 'Return on Equity '
fcf_column = 'Free Cash Flow per Share'
ratio_data = data[[pe_column, pb_column, roa_column, roe_column, fcf_column]]

print(ratio_data)

# Process Return Data
returns_data = get_historical_returns(ticker, start_date, end_date)
# print(returns_data.index) # DateTime Index(Check)

adjusted_ratio_data = repeat_quaterly_data(ratio_data, returns_data)
# print(adjusted_ratio_data)

features = pd.concat([adjusted_ratio_data, returns_data],axis=1)
print(features)

# Scale the data
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_data = scaler.fit_transform(data)

# Create Sequence: use previous 4 quarters to predict the next quarter
seq_length = 3
X, y = create_sequences(features, returns_data, seq_length)
#print(X)


FileNotFoundError: ignored

In [ ]:
#LSTM Model Set Up

# Model architecture
model = tf.keras.Sequential([
    LSTM(512, return_sequences=True),
    Dropout(0.02),
    LSTM(256, return_sequences=True),
    LSTM(128),
    Dense(1, activation='linear', kernel_regularizer=l2(0.0005))
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Split data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train the model
# model.fit(X_train, y_train, epochs=50, batch_size=12, validation_data=(X_test, y_test))

model.fit(X_train, y_train, epochs=500, batch_size=12, validation_data=(X_test, y_test))


Epoch 1/500


5/5 [==============================] - 3s 183ms/step - loss: 0.0109 - val_loss: 0.0083
Epoch 2/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0095 - val_loss: 0.0093
Epoch 3/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0087 - val_loss: 0.0067
Epoch 4/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0084 - val_loss: 0.0069
Epoch 5/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0083 - val_loss: 0.0075
Epoch 6/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0086 - val_loss: 0.0073
Epoch 7/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0083 - val_loss: 0.0092
Epoch 8/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0085 - val_loss: 0.0076
Epoch 9/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0084 - val_loss: 0.0066
Epoch 10/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0084 - val_loss: 0.0068
Epoch 11/500
5/5 [============

In [ ]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)

# Make predictions
predictions = model.predict(X_train)
print(predictions)
print(y_train)

# Inverse transform to get actual values
# predictions = scaler.inverse_transform(predictions)

2/2 [==============================] - 1s 5ms/step
[[-0.02976815]
 [-0.00557859]
 [ 0.05310272]
 [-0.06567945]
 [-0.01677538]
 [ 0.12094283]
 [-0.00459953]
 [-0.03374649]
 [ 0.17504488]
 [-0.00942314]
 [-0.02775423]
 [-0.20159672]
 [-0.12832619]
 [ 0.05523112]
 [ 0.01672073]
 [ 0.06558037]
 [ 0.00352288]
 [-0.03811009]
 [ 0.05324191]
 [ 0.07970035]
 [-0.05013729]
 [ 0.09810044]
 [ 0.0969149 ]
 [ 0.05448768]
 [ 0.09517135]
 [ 0.03597102]
 [-0.11787482]
 [-0.07462232]
 [ 0.13516773]
 [ 0.07979521]
 [ 0.1483014 ]
 [ 0.164749  ]
 [ 0.22758998]
 [-0.11498542]
 [-0.06646731]
 [ 0.07726312]
 [ 0.11302453]
 [-0.03252364]
 [-0.08698383]
 [ 0.01112514]
 [ 0.06503916]
 [-0.0558224 ]
 [ 0.1036368 ]
 [ 0.05581478]
 [-0.02529306]
 [-0.02015727]
 [-0.00151617]
 [ 0.05984043]
 [ 0.01575242]
 [ 0.02078176]
 [-0.02447437]
 [ 0.01648347]
 [-0.00161542]
 [-0.06224626]
 [-0.11332576]
 [ 0.16674586]]
[-0.01524591 -0.01063643  0.0638476  -0.05805073 -0.01501969  0.13076365
 -0.00941828  0.02798332  0.1962268